In [ ]:
!pip install pyngrok flask flask-cors

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-48' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-908967043.py", lin

In [ ]:
from flask import Flask
from pyngrok import ngrok

# 1. SET YOUR TOKEN
# Paste the key you copied from the dashboard here
NGROK_TOKEN = "32ftZp6C53SqZwx6xVMhRWsfOL2_2fZo99y3qp3rqRFLUL5UC"
ngrok.set_auth_token(NGROK_TOKEN)


In [ ]:
import os
from pyngrok import ngrok

# Force kill any lingering ngrok processes in Linux
os.system("killall ngrok")

# Kill via the python wrapper
ngrok.kill()

print("💀 Old Ngrok processes killed. Try running the server cell again now.")


💀 Old Ngrok processes killed. Try running the server cell again now.


In [ ]:
# @title
# 1. FORCE INSTALL CORRECT VERSION (Run this line first!)
# We need version 1.0+ to use the file uploader correctly
!pip install "gradio_client>=1.3.0" fastapi nest-asyncio pyngrok python-multipart uvicorn --upgrade

import uvicorn
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import Response
from pyngrok import ngrok
import nest_asyncio
from gradio_client import Client, handle_file # This import will work now
import shutil
import os
import threading

# --- CONNECT TO OOTDiffusion ---
MODEL_ID = "levihsu/OOTDiffusion"

print(f"⏳ Connecting to {MODEL_ID}...")
try:
    ai_client = Client(MODEL_ID)
    print("✅ SUCCESS: Connected to OOTDiffusion.")
except Exception as e:
    print(f"❌ ERROR: Could not connect to AI. {e}")
    ai_client = None

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all websites to connect
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],
)

@app.post("/try-on")
async def generate_try_on(person_image: UploadFile = File(...), garment_image: UploadFile = File(...)):
    if ai_client is None:
        return Response(content=b"AI Model is offline", status_code=500)

    print("Received request... Uploading images to AI...")

    # 1. Save uploaded files locally
    with open("person.jpg", "wb") as buffer:
        shutil.copyfileobj(person_image.file, buffer)
    with open("garment.jpg", "wb") as buffer:
        shutil.copyfileobj(garment_image.file, buffer)

    try:
        # 2. SEND TO OOTDiffusion
        # FIX: We use handle_file() to ensure the images are actually uploaded
        result = ai_client.predict(
                vton_img=handle_file("person.jpg"), # <--- This fixes the error
                garm_img=handle_file("garment.jpg"), # <--- This fixes the error
                n_samples=1,
                n_steps=20,
                image_scale=2,
                seed=-1,
                api_name="/process_hd"
        )

        # 3. GET RESULT
        # OOTDiffusion returns a list like [{'image': path, ...}]
        if isinstance(result, list) and len(result) > 0:
            if isinstance(result[0], dict):
                 final_path = result[0]['image']
            else:
                 final_path = result[0]
        else:
            final_path = result

        print(f"✅ Generation Complete. File at: {final_path}")

        with open(final_path, "rb") as f:
            img_data = f.read()
        return Response(content=img_data, media_type="image/png")

    except Exception as e:
        print(f"❌ AI Generation Failed: {e}")
        return Response(content=f"AI Error: {str(e)}".encode(), status_code=500)

# --- START SERVER ---
def run_server():
    nest_asyncio.apply()
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)
    server.run()

# Reset Ngrok
ngrok.kill()
tunnel = ngrok.connect(8000)
print(f"✅ NEW PUBLIC URL: {tunnel.public_url}")

thread = threading.Thread(target=run_server)
thread.start()

⏳ Connecting to levihsu/OOTDiffusion...
Loaded as API: https://levihsu-ootdiffusion.hf.space
✅ SUCCESS: Connected to OOTDiffusion.
✅ NEW PUBLIC URL: https://bef86bd0794d.ngrok-free.app
